### import 및 기초작업

In [109]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
warnings.simplefilter("ignore")
pd.options.display.float_format = '{:,.0f}'.format

def get_previous_weekday_yymmdd(date):
    """날짜를 받아 이전 마지막 평일 날짜 yymmdd return"""
    # 전일
    date = date - timedelta(days=1)
    # 전일부터 하루씩 차감하면서 평일이면 return
    while date.weekday() > 4:  # 월요일은 0이고, 금요일은 4
        date -= timedelta(days=1)
    return date.strftime('%y%m%d')

def 파일명(폴더:str, 키워드:str, 부정키워드="") :
    file_list = os.listdir(폴더)
    if 부정키워드 == "" : 
        fn = [file for file in file_list if re.search(키워드, file)]
    else : 
        fn = [file for file in file_list if (re.search(키워드, file)!=None) & (re.search(부정키워드,file)==None)]
    if len(fn) == 1 :
        return join(폴더,fn[0])
    else :
        print("조건을 만족하는 파일이 둘 이상입니다.")
        
credit_deposit_dtype = {"No.":str, "채무자키" : str, "계좌키":str, "담당자키":str, "타입금키":str, "납입회차":str, "계좌번호":str}
account_dtype = {'채무자키':str, '계좌키':str, '계좌번호':str}

In [7]:
################################
company = "솔림" # 솔림 or 대성
workday = "" # 특별히 원하는 날짜 없으면 빈문자열로 남겨둘것
################################

# 입금일(최근 평일 or 선택일)
if workday == "" : 
    # 오늘 이전 중 가장 가까운 평일 날짜 가져오기
    workday = get_previous_weekday_yymmdd(datetime.today())
    
year = "20"+workday[:2] # 2024

# 경로 및 파일명 : 키매칭 파일, 계좌 
wd = join(r"D:\3.자산\신용회복\신용회복 to 일반입금", company, year)
fn_credit_deposit = workday+"_일괄입금구분.xlsx"
fn_account = workday+"_계좌.xlsx"

### 재작업 시작점

In [119]:
cre_dep_ori = pd.read_excel(join(wd, fn_credit_deposit), dtype = credit_deposit_dtype).fillna("")
account_ori = pd.read_excel(join(wd, fn_account), dtype = account_dtype).fillna("")

In [ ]:
cre_dep = cre_dep_ori.copy()
account = account_ori.copy()[["계좌키", "최초원금", "현재원금", "현재법비용", "현재미수이자", "현재연체이자", "현재합계"]]
# 입금파일 계좌키,납입회차 순으로 정렬
cre_dep.sort_values(["계좌키", "납입회차"], inplace=True)

# '계좌키' 열에서 각 값의 누적 빈도를 계산하여 새로운 열 '계좌키_누적빈도'에 추가
cre_dep['계좌키빈도'] = cre_dep.groupby('계좌키').cumcount() + 1

# 최초금액 및 현재금액 병합 - 인덱스 새로 부여됨
cre_dep = cre_dep.merge(account, how='left')
# 변수에 담으면 인덱스가 재설정되긴 하던데, 혹시 모르니 확실하게. 아래 반복문에서 i-1을 하려면 필수!
cre_dep.reset_index(drop=True, inplace=True)

# 이자채권 : 최초원금 10원미만 직접입력             ############################
cre_dep["입금순서"] = np.where(cre_dep.최초원금<10, "1법비용 2미수 3연체 4원금", cre_dep.입금순서)

                                                 ############################
# 오류열 만들고 이미 오류난 거 체크
cond = (cre_dep.현재원금<0) | (cre_dep.현재법비용<0) | (cre_dep.현재미수이자<0) | (cre_dep.현재연체이자<0)
cre_dep["오류"] = np.where(cond, "오류", "")
for i, v in tqdm(cre_dep.iterrows(), total = len(cre_dep)) : 
    # 단건 또는 다건 중 가장 앞선 회차
    if v.계좌키빈도 == 1 :
        # 입금순서 : 분배는 나우리로 하므로 합계만 따지면 됨
        if v.입금순서 != "직접입력" :
            # 오류 체크
            if v.입금합계 > v.현재합계 : cre_dep.loc[i, "오류"] = "오류"
            else :
                # 입금세부항목은 빈칸으로
                cre_dep.loc[i,"입금원금":"입금연체이자"] = ""
                
        # 직접입력 : 입금미수이자와 연체이자는 나우리가 자동으로 나눈다. 혹시 -값이 있는지만 확인
        else : 
            # 오류 체크
            if (v.입금원금 > v.현재원금) or (v.입금미수이자 > v.현재미수이자) or (v.입금연체이자 > v.현재연체이자) : 
                cre_dep.loc[i, "오류"] = "오류"
            else : pass
    # 다건 중 나중 회차 
    # 현재금액 = 전회차현재금액 - 전회차 입금액으로 수정필요, 정렬 했으므로 이전행의 값을 불러오면 됨
    else : 
        # 입금순서 : 분배는 나우리로 하므로 합계만 따지면 됨
        if v.입금순서 != "직접입력" : 
            # 현재합계만 수정
            실현재합계 = cre_dep.loc[i-1, "현재합계"] - cre_dep.loc[i-1, "입금합계"]
            cre_dep.loc[i, "현재합계"] = 실현재합계
            # 오류 체크
            if v.입금합계 > 실현재합계 : cre_dep.loc[i, "오류"] = "오류"
            else :
                # 입금세부항목은 빈칸으로
                cre_dep.loc[i,"입금원금":"입금연체이자"] = ""
        else :
            # 현재금액들 수정, 비용은 직접입력시 차감되지 않는다.
            실현재원금 = cre_dep.loc[i-1, "현재원금"] - cre_dep.loc[i-1, "입금원금"]
            실현재미수이자 = cre_dep.loc[i-1, "현재미수이자"] - cre_dep.loc[i-1, "입금미수이자"]
            실현재연체이자 = cre_dep.loc[i-1, "현재연체이자"] - cre_dep.loc[i-1, "입금연체이자"]
            실현재합계 = cre_dep.loc[i-1, "현재합계"] - cre_dep.loc[i-1, "입금합계"]
            
            cre_dep.loc[i, ["현재원금", "현재미수이자", "현재연체이자", "현재합계"]] = [실현재원금, 실현재미수이자, 실현재연체이자, 실현재합계] # 리스트 아니어도 됨
            
            # 미수와 연체 재분배(체크할 오류x), 원금이나 연체이자에서 음수가 나오면 전산 자체를 수정해야
            if 실현재미수이자 - v.입금미수이자 < 0 : 
                cre_dep.loc[i, "입금미수이자"] = 실현재미수이자 # 실제 남은 현재미수이자만큼만 입금, 나머지는 연체이자로
                cre_dep.loc[i, "입금연체이자"] = v.입금연체이자 - (v.입금미수이자 - 실현재미수이자) # 음수만큼은 연체이자에서 차감
            else : pass

            # 오류 체크 : 원금, 연체이자(입금연체이자 수정됐을 수 있으니 loc로)
            if (v.입금원금 > 실현재원금) or (cre_dep.loc[i, "입금연체이자"] > 실현재연체이자) : 
                cre_dep.loc[i, "오류"] = "오류"
            else : pass
            
# 입금합계 재작성 후 원본과 비교
cre_dep["입금합계"] = cre_dep.입금원금 + cre_dep.입금미수이자 + cre_dep.입금연체이자
원본입금합계 = cre_dep_ori.입금합계.sum()
수정본입금합계 = cre_dep.입금합계.sum()
print(f'원본:{원본입금합계} - 수정본:{수정본입금합계} = {원본입금합계 - 수정본입금합계}')

In [106]:
# 파일 저장 후 오류건 전산 수정 > 계좌 다시 다운
오류건수 = len(cre_dep.query('오류 != "" '))
if 오류건수 > 0 : 
    cre_dep.to_excel(join(wd, workday + "_오류체크.xlsx"))
else : 
    # 일반입금파일 작성
    cols = ["채무자키", "계좌키", "입금일", "입금액", "입금구분", "입금자", "입금자구분", 
            "입금은행", "입금계좌번호", "입금순서", "입금메모", "타입금키"]
    일반입금 = cre_dep[cols]
    일반입금.to_excel(join(wd, workday + "_일반입금변형.xlsx"))
    ##### 타입금키 읽히는지, 읽히지 않더라도 등록되는지 체크해봐라

'240520'